# Connecting python to SQL
We need a library or package to connect them

In [1]:
import pymysql as pms
from sqlalchemy import create_engine
import pandas as pd
import getpass

In [2]:
mysql_pw = getpass.getpass()

········


In [3]:
connection_string = 'mysql+pymysql://root:' + mysql_pw + '@127.0.0.1:3306/bank'
engine = create_engine(connection_string)

In [5]:
df = pd.read_sql_query('Select * from loan', engine)

In [6]:
df

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A
...,...,...,...,...,...,...,...
677,4989,105,981205,352704,48,7348.0,C
678,5221,1284,981205,52512,12,4376.0,C
679,6402,6922,981206,139488,24,5812.0,C
680,5346,1928,981206,55632,24,2318.0,C


## Querying

In [12]:
loan_query = '''SELECT 
    d.A2 AS district,
    SUM(l.amount) AS loans,
    CEILING(AVG(l.amount)) AS avg_loan,
    SUM(l.loan_id) AS num_loans
FROM
    account AS a
        JOIN
    district AS d ON a.district_id = d.A1
        JOIN
    loan AS l USING (account_id)
GROUP BY d.A2;'''
loans = pd.read_sql_query(loan_query, engine)
loans

,district,loans,avg_loan,num_loans
0,Sokolov,148524.0,74262,12217.0
1,Nachod,1768380.0,294730,36827.0
2,Jicin,851364.0,121624,43157.0
3,Pribram,1299192.0,162399,51197.0
4,Hl.m. Praha,12932412.0,153958,533398.0
...,...,...,...,...
72,Blansko,1191024.0,170147,40071.0
73,Domazlice,397008.0,198504,14189.0
74,Liberec,971292.0,138756,43455.0
75,Sumperk,829644.0,165929,34487.0


**Alternative method:**  
df = engine.execute(select * or create or replace view...)

In [36]:
query = '''SELECT * 
FROM trans AS t LEFT JOIN loan AS l USING(account_id) 
WHERE l.status in ('A', 'C')'''
data = pd.read_sql_query(query, engine)

In [37]:
data

,account_id,trans_id,date,type,operation,amount,balance,k_symbol,bank,account,loan_id,date,amount,duration,payments,status
0,5270,1548750,930114,PRIJEM,PREVOD Z UCTU,44749.0,45549.0,,IJ,80269753,6077,931122,79608,24,3317.0,A
1,11265,3393738,930114,PRIJEM,VKLAD,1000.0,1000.0,,,0,7284,930915,52788,12,4399.0,A
2,10364,3122924,930117,PRIJEM,VKLAD,1100.0,1100.0,,,0,7121,931110,21924,36,609.0,A
3,3834,1121963,930119,PRIJEM,VKLAD,700.0,700.0,,,0,5754,940928,23052,12,1921.0,A
4,9307,2809952,930124,PRIJEM,VKLAD,900.0,900.0,,,0,6895,940919,41904,12,3492.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136882,2872,3625495,981231,PRIJEM,,67.9,17036.9,UROK,,0,5556,970519,268320,60,4472.0,C
136883,2910,3626751,981231,PRIJEM,,187.2,42249.6,UROK,,0,5561,980829,437460,60,7291.0,C
136884,2933,3627577,981231,PRIJEM,,132.0,31046.4,UROK,,0,5568,940517,272520,60,4542.0,C
136885,2912,3626847,981231,PRIJEM,,271.6,58694.2,UROK,,0,5563,960229,380160,60,6336.0,C


In [38]:
loans_status_year = '''SELECT 
    YEAR(date) AS year, status, COUNT(DISTINCT loan_id) AS loans
FROM
    loan
GROUP BY year, status;'''
loans_year = pd.read_sql_query(loans_status_year, engine)
loans_year

,year,status,loans
0,1993,A,16
1,1993,B,4
2,1994,A,73
3,1994,B,12
4,1994,C,14
5,1994,D,2
6,1995,A,45
7,1995,B,6
8,1995,C,33
9,1995,D,6


In [41]:
def get_AC_loans(engine):
    import pandas as pd
    query = '''SELECT * 
FROM trans AS t LEFT JOIN loan AS l USING(account_id) 
WHERE l.status in ('A', 'C')'''
    data = pd.read_sql_query(query, engine)
    return data
get_AC_loans(engine)

,account_id,trans_id,date,type,operation,amount,balance,k_symbol,bank,account,loan_id,date,amount,duration,payments,status
0,5270,1548750,930114,PRIJEM,PREVOD Z UCTU,44749.0,45549.0,,IJ,80269753,6077,931122,79608,24,3317.0,A
1,11265,3393738,930114,PRIJEM,VKLAD,1000.0,1000.0,,,0,7284,930915,52788,12,4399.0,A
2,10364,3122924,930117,PRIJEM,VKLAD,1100.0,1100.0,,,0,7121,931110,21924,36,609.0,A
3,3834,1121963,930119,PRIJEM,VKLAD,700.0,700.0,,,0,5754,940928,23052,12,1921.0,A
4,9307,2809952,930124,PRIJEM,VKLAD,900.0,900.0,,,0,6895,940919,41904,12,3492.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136882,2872,3625495,981231,PRIJEM,,67.9,17036.9,UROK,,0,5556,970519,268320,60,4472.0,C
136883,2910,3626751,981231,PRIJEM,,187.2,42249.6,UROK,,0,5561,980829,437460,60,7291.0,C
136884,2933,3627577,981231,PRIJEM,,132.0,31046.4,UROK,,0,5568,940517,272520,60,4542.0,C
136885,2912,3626847,981231,PRIJEM,,271.6,58694.2,UROK,,0,5563,960229,380160,60,6336.0,C
